In [1]:
import pandas as pd
import numpy as np
import datetime
import time

In [2]:
df = pd.read_csv('./../Data/boardcomment/boardcomment_kia.csv',dtype={'com_code': np.str,'comment': np.str})
df_date = pd.read_csv('./../Data/주가_휴장일추출.csv')

C:\Users\seon\AppData\Local\Temp\ipykernel_25176\1908212109.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = pd.read_csv('./../Data/boardcomment/boardcomment_kia.csv',dtype={'com_code': np.str,'comment': np.str})


In [3]:
df.rename(columns={'com_code':'code','comment':'content','decommend':'unlike','recommend':'like','view':'view_count'},inplace=True)
df=df[['code', 'title', 'content', 'date', 'like', 'unlike','view_count']]

In [4]:
df.head()

,code,title,content,date,like,unlike,view_count
0,000270,공매도 이기고 가면 올라갈 수는 있음 ㅋ,어찌하냐?,2022.12.21 14:29,1,0,564
1,000270,기아 다시 9만 가즈아아아아,가즈아 가즈아 애국기아 민노 out,2022.12.21 14:20,5,2,664
2,000270,기관과 외국인 지들끼리 놀아라 하고,개미는 발빼자 개미할배는 발뺀지 오래다,2022.12.21 14:29,1,1,569
3,000270,"앞으로 사재기 없다, 공장 지을 때 참고 하세요.",it's this both of all i 'm very consider\nthe ...,2022.12.21 14:29,1,0,588
4,000270,외국인들,외국인들\n어제 사는 척 개미 꼬시기 해놓고\n오늘 거래량도 없이 매도하는 거 봐라...,2022.12.21 14:36,2,0,598


In [5]:
df.shape

(149210, 7)

In [6]:
df.dropna(inplace=True)

In [7]:
df.shape

(149208, 7)

In [8]:
# 최소한의 반응이 있는 글들 사용
# 좋아요 싫어요 합이 1 이상
df = df.loc[(df['like']+df['unlike'])>=1]
df.reset_index(drop=True,inplace=True)
df.shape

(110964, 7)

In [9]:
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x))
df_date = df_date['날짜']
df_date = df_date.apply(lambda x: pd.to_datetime(str(x)))

In [10]:
df['content'] = df['content'].apply(lambda x : x.replace('\n',''))

In [11]:
idx = np.where((df['date']>datetime.datetime(2017, 11, 30, 17, 50))&(df['date']<datetime.datetime(2022, 11, 30, 17,50)))

In [12]:
df = df.iloc[idx]
df.reset_index(drop=True,inplace=True)

In [13]:
idx1 = np.where((df_date>=datetime.datetime(2017, 12, 1))&(df_date<=datetime.datetime(2022, 11, 30)))

In [14]:
df_date = df_date[np.min(idx1)-1:np.max(idx1)+2]
df_date.reset_index(drop=True,inplace=True)


In [16]:
def date_category(x):
    for i in range(len(df_date)-1):
        if ((df_date[i] + datetime.timedelta(hours=17,minutes=50)) <= x) & (x<=(df_date[i+1] + datetime.timedelta(hours=17,minutes=49))):
            return df_date[i+1].strftime('%Y-%m-%d')

In [17]:
df['date_cate']= df['date'].apply(lambda x: date_category(x))

In [23]:
df.to_csv('./../Data/boardcomment/pre_comment/comment_kia.csv',encoding='utf-8-sig')